# knn op vlak van pageviews (WIP)

In [56]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, select, MetaData
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
import sklearn as sk
import numpy as np

In [ ]:
load_dotenv()
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)


In [58]:
#get Inschrijvingen, Contact en account
print(metadata.tables.keys())
Pageviews = metadata.tables['Pageviews']
Contact = metadata.tables['Contactfiche']
Account = metadata.tables['Account']
query = select(Pageviews.c.PageTitle, Account.c.AccountId).join(Contact, Pageviews.c.ContactId == Contact.c.ContactpersoonId).join(Account, Contact.c.AccountId == Account.c.AccountId)
result = conn.execute(query)

#convert to dataframe
df = pd.DataFrame(result.fetchall())


dict_keys(['Account', 'AccountActiviteitscode', 'Activiteitscode', 'AccountFinancieleData', 'AfspraakAccount', 'AfspraakContact', 'Contactfiche', 'Persoon', 'AfspraakVereistContact', 'Campagne', 'ContactficheFunctie', 'Functie', 'Gebruiker', 'InfoEnKlachten', 'Inschrijving', 'Mailing', 'Pageviews', 'Visits', 'SendEmailClicks', 'Sessie', 'SessieInschrijving', 'sysdiagrams', 'WebContent'])


In [59]:
print(df.head())

                                           PageTitle  \
0  Weer hogere loonindexering op komst in 2024 | ...   
1                     Nieuwjaarsinvitito 2023 | Voka   
2                               Netwerkevents | Voka   
3                                   Marketing | Voka   
4   Bedrijfsafval: dit verandert op 1 januari | Voka   

                              AccountId  
0  99C841DF-C768-E111-B43A-00505680000A  
1  44EDC779-6A76-E611-80E3-001DD8B72B62  
2  371D1C8C-D868-E111-B43A-00505680000A  
3  FCA86F9D-C668-E111-B43A-00505680000A  
4  6D3D3D4D-B067-E111-A00F-00505680000A  


In [60]:
#make a dataframe of all the page titles per account
def make_dataframe_per_account(df):
    account_list = df['AccountId'].unique()
    pages_list = []
    for i, account in enumerate(account_list):
        pages = df[df['AccountId'] == account]['PageTitle'].unique().tolist()
        pages_list.append(pages)
    return account_list, pages_list

In [61]:
accounts, pages = make_dataframe_per_account(df)
print(accounts)

['99C841DF-C768-E111-B43A-00505680000A'
 '44EDC779-6A76-E611-80E3-001DD8B72B62'
 '371D1C8C-D868-E111-B43A-00505680000A' ...
 'DB5B61D6-C468-E111-B43A-00505680000A'
 '8180C9B5-C668-E111-B43A-00505680000A'
 'ECC4C144-5AC8-EB11-8123-001DD8B72B62']


In [62]:
print(pages[0].info)

AttributeError: 'list' object has no attribute 'info'

In [89]:
#function that appends all pages to a list and remove doubles
def create_pages_list(pages) -> list:
    return list(set(pages))

In [90]:
def remove_None_and_Zoekresultaten(pages) -> list:
    all_pages = []
    for page in pages:
        pages = [page for page in pages if page is not None]
        for p in page:
            if p is not None and 'Zoekresultaten' not in p:
                all_pages.append(p) 
    return all_pages

In [93]:
#remove None values and values with Zoekresultaten in it
prepared_pages = remove_None_and_Zoekresultaten(pages)
all_pages = create_pages_list(prepared_pages)
print(all_pages)
print(len(all_pages))
print(len(prepared_pages))

In [81]:
#print all pages containing the word 'zoekresultaten'
#first drop None values
all_pages = [page for page in all_pages if page is not None]
zoekresultaten = [page for page in all_pages if 'Zoekresultaten' in page]
print(zoekresultaten)
print(len(zoekresultaten))
print(len(all_pages))

[]
0
14699


In [84]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
mlb.fit([all_pages])
print(mlb.classes_)
print(len(mlb.classes_))


['" Hitting the ground running" | Voka'
 '"2.330 zonnepanelen besparen 432 ton CO2" | Voka'
 '"2023 vertoont eerste tekenen van lichte vooruitgang" | Voka' ...
 "wisselkoersrisico's" 'working paper' '| Voka']
14699


In [87]:
#remove none values from pages
pages = [page for page in pages if page is not None]
print(pages[0])
mlb.fit_transform(pages)

['Weer hogere loonindexering op komst in 2024 | Voka', 'Coronavirus: de meest gestelde vragen over de impact op je onderneming | Voka', 'Nieuws - Voka nationaal | Voka', 'Overnemen & overlaten | Voka', 'Netwerkevents | Voka', 'Trajecten | Voka', '?Environmental, Social and Governance worden steeds belangrijker? | Voka', 'Activiteiten | Voka', 'Activiteiten - Mechelen-Kempen | Voka', 'Ondernemers in de Brusselse Metropool 2022 #1 | Voka', 'Hoe bescherm ik de innovaties van mijn onderneming? | Voka', 'Hoe zit het programma van het Familio-traject eruit? | Voka', 'Kantoren Mechelen-Kempen | Voka', 'Andreas Pfeffer volgt in 2023 Jo De Backer op | Voka', 'Voka Tribune Januari 2022 - In 2022 gaan we samen groeien | Voka', 'Nee, er is geen lerarentekort | Voka', 'VCDO over de grens | Voka', 'Zoekresultaten voor "iftar" | Voka', 'Layout-mogelijkheden website | Voka', 'Inschrijven voor Welt Mechelen-Kempen | Voka', 'Oorlog in Oekraïne: de meest gestelde vragen | Voka', 'Brexit: de meest gesteld

TypeError: '<' not supported between instances of 'NoneType' and 'str'